In [ ]:
from src.Camera import Camera
from src.Data import Data
from src.VisualOdometry import VisualOdometry
from src.utils import *
from src.data_association import *
import numpy as np
import plotly.graph_objects as go   
import matplotlib.pyplot as plt

In [ ]:
vo = VisualOdometry()
vo.initialize()

In [ ]:
pose_0 = vo.get_trajectory()[0]
pose_1 = vo.get_trajectory()[1]

world_points = vo.get_map()['position']
print('Number of world points: ', len(world_points))

x_coords = [point[0] for point in world_points]
y_coords = [point[1] for point in world_points]
z_coords = [point[2] for point in world_points]

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=2)))

fig.add_trace(go.Scatter3d(x=[pose_0[0, 3]], y=[pose_0[1, 3]], z=[pose_0[2, 3]], mode='markers', marker=dict(size=4, color='red')))
fig.add_trace(go.Scatter3d(x=[pose_1[0, 3]], y=[pose_1[1, 3]], z=[pose_1[2, 3]], mode='markers', marker=dict(size=4, color='blue')))
fig.add_trace(go.Scatter3d(x=[pose_0[0, 3], pose_1[0, 3]], y=[pose_0[1, 3], pose_1[1, 3]], z=[pose_0[2, 3], pose_1[2, 3],], mode='lines', line=dict(color='black', width=2)))

fig.show()

In [ ]:
NUM_FRAMES = 30
for i in range(1,NUM_FRAMES+1):
    print(f'i: {i}')
    vo.update(i)
    print("################################## \n")

In [ ]:
estimated_trajectory = vo.get_trajectory()
gt_trajectory = vo.get_data().get_trajectory_data()

C = vo.get_camera().get_camera_transform()
 
estimated_trajectory_in_world = [] 

for i in range(len(estimated_trajectory)):
    pose = estimated_trajectory[i]
    pose_in_world = C @ pose
    estimated_trajectory_in_world.append(pose_in_world)

    print(f'Pose {i}\n{np.round(pose,2)}\nPose in world {i}\n{np.round(pose_in_world,2)}\n\n')

estimated_positions = []
estimated_positions_in_world = [] 
for i in range(len(estimated_trajectory)):
    estimated_positions.append(estimated_trajectory[i][:3, 3])
    estimated_positions_in_world.append(estimated_trajectory_in_world[i][:3, 3])

gt_poses = []
gt_positions = []
for i in range(1,NUM_FRAMES+1):
    x_gt, y_gt, theta_gt = gt_trajectory[i]
    gt_poses.append(np.array([x_gt, y_gt, 0, 0, 0, theta_gt]))
    gt_positions.append(np.array([x_gt, y_gt, 0]))
    
fig = go.Figure()
scale = 0.208
gt_x_coords = [position[0] for position in gt_positions]
gt_y_coords = [position[1] for position in gt_positions]
gt_z_coords = [position[2] for position in gt_positions]
fig.add_trace(go.Scatter3d(x=gt_x_coords, y=gt_y_coords, z=gt_z_coords, mode='lines', name='GT trajectory', line=dict(color='green')))

estimated_x_coords = [position[0]*scale for position in estimated_positions]
estimated_y_coords = [position[1]*scale for position in estimated_positions]
estimated_z_coords = [position[2]*scale for position in estimated_positions]
fig.add_trace(go.Scatter3d(x=estimated_x_coords, y=estimated_y_coords, z=estimated_z_coords, mode='lines', name='Estimated trajectory', line=dict(color='red')))

estimated_x_coords_in_world = [position[0]*scale for position in estimated_positions_in_world]
estimated_y_coords_in_world = [position[1]*scale for position in estimated_positions_in_world]
estimated_z_coords_in_world = [position[2]*scale for position in estimated_positions_in_world]
fig.add_trace(go.Scatter3d(x=estimated_x_coords_in_world, y=estimated_y_coords_in_world, z=estimated_z_coords_in_world, mode='lines', name='Estimated trajectory in world frame', line=dict(color='blue')))

# set the scale of the plot fixed equal for all axis
fig.update_layout(scene=dict(aspectmode='data'))




fig.show()

In [ ]:
index = 17

data = vo.get_data()
camera = vo.get_camera()
trajectory = vo.get_trajectory()[index]

camera.set_c_T_w(np.linalg.inv(trajectory))

measurement = data.get_measurements_data_points(index)

# matches = data_association_2Dto3D(measurement, vo.get_map(), camera)
matches = data_association_on_appearance(measurement, vo.get_map())

image_points = matches['points_1']
world_points = matches['points_2']

projected_world_points = []
for i in range(len(world_points)):
    world_point = world_points[i]
    is_inside, projected_image_point = camera.project_point(world_point)
    if is_inside: projected_world_points.append(projected_image_point)

fig, ax = plt.subplots()
ax.imshow(np.ones((480, 640, 3)))
ax.scatter([point[0] for point in image_points], [point[1] for point in image_points], color='green', marker='o')
ax.scatter([point[0] for point in projected_world_points], [point[1] for point in projected_world_points], color='red', marker='x')
plt.grid()
plt.show()

In [ ]:
world_points = vo.get_map()['position']
print('Number of world points: ', len(world_points))

x_coords = [point[0] for point in world_points]
y_coords = [point[1] for point in world_points]
z_coords = [point[2] for point in world_points]

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=2)))
fig.show()

In [ ]:
gt_world_data = data.get_world_data()
triangulated_points = vo.get_map()

matches = data_association_on_appearance(triangulated_points, gt_world_data)
triangulated_points = matches['points_1']
gt_world_points = matches['points_2']

for i in range(len(triangulated_points)):
    point_1 = matches['points_1'][i]
    point_1 = camera.get_camera_transform() @ np.append(point_1, 1)
    point_1 = point_1[:3] / point_1[3]
    point_2 = matches['points_2'][i]

    distance = np.linalg.norm(point_1 - point_2)
    print(f'Distance between point {i}: {np.round(distance, 2)}')


triangulated_points_hom = [np.append(point, 1) for point in triangulated_points]
triangulated_points_world = camera.get_camera_transform() @ np.array(triangulated_points_hom).T


x_coords = [point[0]*0.208 for point in triangulated_points_world.T]  
y_coords = [point[1]*0.208 for point in triangulated_points_world.T]
z_coords = [point[2]*0.208 for point in triangulated_points_world.T]

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=4)))

x_coords_gt = [point[0] for point in gt_world_points]
y_coords_gt = [point[1] for point in gt_world_points]
z_coords_gt = [point[2] for point in gt_world_points]

fig.add_trace(go.Scatter3d(x=x_coords_gt, y=y_coords_gt, z=z_coords_gt, mode='markers', marker=dict(size=2, color='red')))

fig.show()